Installing the required packages

In [ ]:
!pip install gcsfs pandas transformers torch pyarrow

In [ ]:
# Authenticating for access
from google.colab import auth
auth.authenticate_user()


In [ ]:

# Read Parquet file from GCS
gcs_path = "gs://603-yelp-data/processed_reviews.parquet/"


In [ ]:
import pandas as pd
df = pd.read_parquet(gcs_path, engine='pyarrow')

# Display the first few rows
df.head()


,business_id,cool,date,funny,review_id,stars,text,useful,user_id,review_length,review_year,review_month,rating_category
0,BryQISFnkCdcJET-tpAx3w,0,2015-06-20 01:52:39,0,--1z0MjmPFmNQLpnwwhBOA,4.0,I brought my family here recently because I ha...,0,xg36DnPGqxPfGZN1qqXlYg,1490,2015,6,high
1,I_mCFePUG2MGuH-zuws7bA,0,2016-04-13 23:43:53,0,--6RL5X9PiswVnpKKHDB9g,5.0,You really cant beat Great Full Gardens. Every...,0,l2qhjxdNQeEAn3yPj-wN1A,361,2016,4,high
2,p2yZcS_C-TjlEUXrLQzFrg,0,2018-03-25 00:48:47,0,--7f0VAcFgpK3Hz8uUXrFQ,4.0,Excellent service. Wait staff is very attentiv...,0,ATZPb63VYwJXa3biMuJPfA,169,2018,3,high
3,Auc20f0uYTIwPnJIqcdmNw,0,2011-08-07 07:55:00,0,--CQtyFgZgR5LAGspYI4Wg,2.0,I was visiting the area since I'm out of town ...,2,bGSdyBiaARMCdGTJgeMixw,461,2011,8,low
4,2EOinQpyU4ChpyJ4KusO2A,0,2019-03-19 15:56:36,0,--DcWXBx2ei_ME39LrL6Hg,4.0,Good stuff even for the non vegan. I was surpr...,1,11yiaQf8NGEVFjk2egSdmg,132,2019,3,high


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 13 columns):
 #   Column           Dtype  
---  ------           -----  
 0   business_id      object 
 1   cool             int64  
 2   date             object 
 3   funny            int64  
 4   review_id        object 
 5   stars            float64
 6   text             object 
 7   useful           int64  
 8   user_id          object 
 9   review_length    int32  
 10  review_year      int32  
 11  review_month     int32  
 12  rating_category  object 
dtypes: float64(1), int32(3), int64(3), object(6)
memory usage: 613.3+ MB


In [ ]:
# Calculate the length of each review
df['review_length'] = df['text'].apply(lambda x: len(x.split()))

# Display the first few rows to verify the 'review_length' column
print(df[['text', 'review_length']].head())

# Calculate the average review length
average_length = df['review_length'].mean()

print(f"Average review length: {average_length} words")


                                                text  review_length
0  I brought my family here recently because I ha...            274
1  You really cant beat Great Full Gardens. Every...             64
2  Excellent service. Wait staff is very attentiv...             33
3  I was visiting the area since I'm out of town ...             97
4  Good stuff even for the non vegan. I was surpr...             24
Average review length: 104.77632326602082 words


In [ ]:
# Calculate the length of each review (in terms of words)
df['review_length'] = df['text'].apply(lambda x: len(x.split()))

# Total number of reviews
total_reviews = df.shape[0]

# Maximum review length
max_length = df['review_length'].max()

# Average review length
average_length = df['review_length'].mean()

# Print the results
print(f"Total number of reviews: {total_reviews}")
print(f"Maximum review length: {max_length} words")
print(f"Average review length: {average_length:.2f} words")


Total number of reviews: 6990280
Maximum review length: 1070 words
Average review length: 104.78 words


In [ ]:
# Count the number of reviews with a length greater than 512 words
reviews_above_512 = df[df['review_length'] > 512].shape[0]

# Print the result
print(f"Number of reviews longer than 512 words: {reviews_above_512}")


Number of reviews longer than 512 words: 59450


In [ ]:
# Drop reviews longer than 512 words for better analysis
df = df[df['review_length'] <= 512]

# Print the updated number of reviews
print(f"Updated number of reviews: {df.shape[0]}")


Updated number of reviews: 6930830


Sentimental analysis by batch processing

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import pipeline
import torch

# Check if GPU is available and set the device
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, else CPU

In [ ]:


# Initialize the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

# Move the model to GPU (if available)
model = model.to(device)

# Initialize sentiment analysis pipeline with explicit tokenizer and model
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)

# Filter the first 50,000 reviews after dropping reviews longer than 512 words
reviews_to_analyze = df.head(50000)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

sentimental analysis for the first 50,000 records

In [ ]:
from tqdm import tqdm  # Import the tqdm library for progress bars

# Initialize tqdm for interactive processing
tqdm.pandas()

# Function to apply sentiment analysis with tqdm progress
def get_sentiment_with_progress(text):
    # Tokenize the text, truncating and padding it as necessary
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU
    result = model(**inputs)
    label = result.logits.argmax(axis=1).item()  # Get the predicted sentiment
    return 'POSITIVE' if label == 1 else 'NEGATIVE'

# Apply sentiment analysis to reviews with progress tracking
reviews_to_analyze['sentiment'] = reviews_to_analyze['text'].progress_apply(get_sentiment_with_progress)

# Output results
print(f"Total reviews processed for sentiment analysis: {len(reviews_to_analyze)}")


100%|██████████| 50000/50000 [06:34<00:00, 126.73it/s]

Total reviews processed for sentiment analysis: 50000



<ipython-input-12-73ce4f020fd7>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_to_analyze['sentiment'] = reviews_to_analyze['text'].progress_apply(get_sentiment_with_progress)


for the next 100,000 records

In [ ]:
reviews_to_analyze_batch2 = df[50000:150000]


# Initialize tqdm for interactive processing
tqdm.pandas()

# Function to apply sentiment analysis with tqdm progress
def get_sentiment_with_progress(text):
    # Tokenize the text, truncating and padding it as necessary
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU (if available)
    result = model(**inputs)
    label = result.logits.argmax(axis=1).item()  # Get the predicted sentiment
    return 'POSITIVE' if label == 1 else 'NEGATIVE'

# Apply sentiment analysis to reviews with progress tracking
reviews_to_analyze_batch2['sentiment'] = reviews_to_analyze_batch2['text'].progress_apply(get_sentiment_with_progress)

# Output results
print(f"Total reviews processed for sentiment analysis: {len(reviews_to_analyze_batch2)}")

100%|██████████| 100000/100000 [12:51<00:00, 129.57it/s]

Total reviews processed for sentiment analysis: 100000



<ipython-input-13-e5f9d12c219b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_to_analyze_batch2['sentiment'] = reviews_to_analyze_batch2['text'].progress_apply(get_sentiment_with_progress)


for the next 350,000 records

In [ ]:
reviews_to_analyze_batch3 = df[150000:500000]


# Initialize tqdm for interactive processing
tqdm.pandas()  # This integrates tqdm with pandas apply function

# Function to apply sentiment analysis with tqdm progress
def get_sentiment_with_progress(text):
    # Tokenize the text, truncating and padding it as necessary
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU (if available)
    result = model(**inputs)
    label = result.logits.argmax(axis=1).item()  # Get the predicted sentiment
    return 'POSITIVE' if label == 1 else 'NEGATIVE'

# Apply sentiment analysis to reviews with progress tracking
reviews_to_analyze_batch3['sentiment'] = reviews_to_analyze_batch3['text'].progress_apply(get_sentiment_with_progress)

# Output results
print(f"Total reviews processed for sentiment analysis: {len(reviews_to_analyze_batch3)}")

100%|██████████| 350000/350000 [46:52<00:00, 124.47it/s]

Total reviews processed for sentiment analysis: 350000



<ipython-input-14-261c6ef7d958>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_to_analyze_batch3['sentiment'] = reviews_to_analyze_batch3['text'].progress_apply(get_sentiment_with_progress)


for the next 500.000 records

In [ ]:
reviews_to_analyze_batch4 = df[500000:1000000]

from tqdm import tqdm  # Import the tqdm library for progress bars

# Initialize tqdm for interactive processing
tqdm.pandas()  # This integrates tqdm with pandas apply function

# Function to apply sentiment analysis with tqdm progress
def get_sentiment_with_progress(text):
    # Tokenize the text, truncating and padding it as necessary
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU (if available)
    result = model(**inputs)
    label = result.logits.argmax(axis=1).item()  # Get the predicted sentiment
    return 'POSITIVE' if label == 1 else 'NEGATIVE'

# Apply sentiment analysis to reviews with progress tracking
reviews_to_analyze_batch4['sentiment'] = reviews_to_analyze_batch4['text'].progress_apply(get_sentiment_with_progress)

# Output results
print(f"Total reviews processed for sentiment analysis: {len(reviews_to_analyze_batch4)}")

100%|██████████| 500000/500000 [1:07:30<00:00, 123.44it/s]

Total reviews processed for sentiment analysis: 500000



<ipython-input-15-1687f717e119>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_to_analyze_batch4['sentiment'] = reviews_to_analyze_batch4['text'].progress_apply(get_sentiment_with_progress)


save the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Create the directory if it doesn't exist
output_directory = '/content/drive/MyDrive'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [ ]:
# Combine the batches into one final DataFrame
final_df = pd.concat([reviews_to_analyze, reviews_to_analyze_batch2, reviews_to_analyze_batch3, reviews_to_analyze_batch4], ignore_index=True)

# Save the final combined data to Google Drive (CSV format)
final_file_path = '/content/drive/MyDrive/final_reviews_sentiment.csv'
final_df.to_csv(final_file_path, index=False)

print(f"Total reviews processed: {len(final_df)}")
print(f"Final dataset saved to: {final_file_path}")

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from google.cloud import storage
from google.auth import exceptions
import os

# Authenticate with Google Cloud
def authenticate_gcs():
    try:
        client = storage.Client()
        print("Authenticated with Google Cloud Storage successfully.")
        return client
    except exceptions.DefaultCredentialsError:
        print("Authentication failed. Please authenticate using a service account key.")
        raise

# Upload file to GCS
def upload_to_gcs(file_path, bucket_name, destination_blob_name):
    try:
        # Initialize the GCS client
        client = authenticate_gcs()

        # Get the bucket object
        bucket = client.bucket(bucket_name)

        # Check if the bucket exists
        if not bucket.exists():
            print(f"Error: Bucket '{bucket_name}' does not exist.")
            return

        # Create a blob (file) in the bucket
        blob = bucket.blob(destination_blob_name)

        # Upload the local file to GCS
        blob.upload_from_filename(file_path)
        print(f"File '{file_path}' uploaded to gs://{bucket_name}/{destination_blob_name}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Define file paths and GCS paths
file_path = '/content/drive/MyDrive/final_reviews_sentiment.csv'  # Local file path
bucket_name = '603-yelp-data'  # Your GCS bucket name
destination_blob_name = 'final_reviews_sentiment.csv'  # Desired name in GCS

# Upload the file to GCS
upload_to_gcs(file_path, bucket_name, destination_blob_name)


Authenticated with Google Cloud Storage successfully.
File '/content/drive/MyDrive/final_reviews_sentiment.csv' uploaded to gs://603-yelp-data/final_reviews_sentiment.csv


The file with sentimental analysis is mounted uploaded to cloud storage